# Part 2
## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt

# Import Google API Key
from config import g_key

In [2]:
# Store the saved CSV as a new DataFrame.
city_data_df = pd.read_csv("data/database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Mataura,NZ,-46.19,168.86,48.18,72,79,1.32,broken clouds,0.0,0
1,1,Ozernovskiy,RU,51.50,156.52,46.42,77,100,1.63,overcast clouds,0.0,0
2,2,San Patricio,MX,19.22,-104.70,84.20,74,20,9.17,few clouds,0.0,0
3,3,Atkinson,US,42.84,-71.15,81.00,57,1,8.05,clear sky,0.0,0
4,4,Cozma,RO,46.81,24.52,63.00,93,100,3.36,light intensity shower rain,0.0,0


In [3]:
# Ask the customer to add a minimum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Ask the customer to add a maximum temperature value.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Ask the customer if they would like it to be raining.
rain = (input("Do you want it to be raining? (yes/no) "))

# Ask the customer if they would like it to be snowing.
snow = (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria.
if rain =="yes" and snow == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0.0) & \
                                           (city_data_df["Snow (inches)"] == 0)].dropna()

elif rain =="no" and snow == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == 0) & \
                                           (city_data_df["Snow (inches)"] > 0.0)].dropna()
    
elif rain =="no" and snow == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == 0) & \
                                           (city_data_df["Snow (inches)"] == 0)].dropna()   
    
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0.0) & \
                                           (city_data_df["Snow (inches)"] > 0.0)].dropna()        

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,San Patricio,MX,19.22,-104.70,84.20,74,20,9.17,few clouds,0.0,0
3,3,Atkinson,US,42.84,-71.15,81.00,57,1,8.05,clear sky,0.0,0
11,11,Kapaa,US,22.08,-159.32,82.99,69,40,20.80,scattered clouds,0.0,0
21,21,Iralaya,HN,15.00,-83.23,82.40,83,40,5.82,scattered clouds,0.0,0
23,23,Kailua,US,21.40,-157.74,84.20,61,75,14.99,broken clouds,0.0,0
25,25,Kavieng,PG,-2.57,150.80,84.92,72,26,8.72,scattered clouds,0.0,0
27,27,Hasaki,JP,35.73,140.83,80.01,64,20,3.36,few clouds,0.0,0
31,31,Luganville,VU,-15.53,167.17,82.40,74,75,5.82,broken clouds,0.0,0
34,34,Portland,US,45.52,-122.68,86.00,30,1,4.70,clear sky,0.0,0
35,35,Yarada,IN,17.65,83.27,82.40,88,40,2.24,mist,0.0,0


In [5]:
# Create hotel_df with specific columns
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [7]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name. Use a try except to deal with errors
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,San Patricio,MX,84.20,few clouds,19.22,-104.70,Hotel Cabo Blanco
3,Atkinson,US,81.00,clear sky,42.84,-71.15,Atkinson Resort & Country Club
11,Kapaa,US,82.99,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
21,Iralaya,HN,82.40,scattered clouds,15.00,-83.23,Iglesia morava renovada
23,Kailua,US,84.20,broken clouds,21.40,-157.74,Sheffield House Bed & Breakfast
25,Kavieng,PG,84.92,scattered clouds,-2.57,150.80,Nusa Island Retreat
27,Hasaki,JP,80.01,few clouds,35.73,140.83,Inubosaki Kanko Hotel
31,Luganville,VU,82.40,broken clouds,-15.53,167.17,The Espiritu Hotel
34,Portland,US,86.00,clear sky,45.52,-122.68,Portland Marriott City Center
35,Yarada,IN,82.40,mist,17.65,83.27,"D-26/2, Dolphin Hill,"


In [9]:
# Create the CSV output file.
output_data_file = "data/WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file)

In [10]:
# Create info_box_templtate format
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame hotel row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
#Define locations and fig
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add markers on top of the map for each city and hotel
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))